In [23]:
import csv
import ipywidgets as widgets
from IPython.display import display
from datetime import datetime

# Function to generate filename based on current year and week
def generate_filename():
    current_date = datetime.now()
    year = current_date.year
    week_number = current_date.isocalendar()[1]
    filename = f"{year}-{week_number}-Estimation"
    return filename

# Create a file upload widget
upload_widget = widgets.FileUpload(
    accept='.csv',  # Accept only CSV files
    multiple=False  # Single file upload
)

# Create a UI for user input of Won bid
won_bids_input = widgets.IntText(
    value=0,
    description='Won Bids:',
    disabled=False
)

# Create a button to trigger file processing
generate_button = widgets.Button(
    description="Generate File",
    button_style='success'
)

output_label = widgets.Label()

# Function to process the uploaded file and calculate win rate
def process_file(button):
    if not upload_widget.value:
        output_label.value = "Please upload a file before generating."
        return
    # Read the uploaded CSV file
    uploaded_file = list(upload_widget.value)[0]
    # Read the CSV file
    with open(uploaded_file['name'], 'r') as file:
        reader = csv.reader(file)
        rows = list(reader)
    
    # Initialize variables
    todo_bids = []
    current_bids = []
    total_bids = []

    # Process each row
    for row in rows:
        if row[0] == "To Do / Bid":
            for i in range(1, len(row), 2):
                if row[i] and not row[i].startswith("http"):
                    todo_bids.append((row[i], row[i+1]))
        elif row[0] == "In Progress - Estimating":
            for i in range(1, len(row), 2):
                if row[i] and not row[i].startswith("http"):
                    current_bids.append((row[i], row[i+1]))
        elif row[0].startswith("Submitted") and "2024" in row[0]:
            for i in range(1, len(row), 2):
                if row[i] and not row[i].startswith("http"):
                    total_bids.append((row[i], row[i+1]))

    # Calculate the number of bids
    num_todo_bids = len(todo_bids)
    num_current_bids = len(current_bids)
    num_total_bids = len(total_bids)

    # Get the number of won bids from the user
    won_bids = won_bids_input.value

    # Calculate the win rate
    win_rate = (won_bids / num_total_bids) * 100 if num_total_bids > 0 else 0

    # Write the results to a text file
    file_name = generate_filename()
    with open(file_name + '.txt', 'w') as file:
        file.write(f"Number of Todo Bids: {num_todo_bids}\n")        
        file.write(f"\nNumber of Current Bids: {num_current_bids}\n")
        file.write(f"\nTotal Bids: {num_total_bids}\n")  
        file.write(f"\nWon Bids: {won_bids}\n")
        file.write(f"Win Rate: {win_rate:.2f}%\n")

    print(f"The summary has been written to {file_name}.txt.")

# Attach the event handler to the button
generate_button.on_click(process_file)

# Display the UI elements
display(widgets.VBox([
    widgets.Label("Upload a CSV file and enter the number of Won Bids to generate the summary:"),
    upload_widget,
    won_bids_input,
    generate_button,
    output_label
]))